<a href="https://colab.research.google.com/github/saakolch/procedure_of_extracting_data/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**We have to predict the patient condition based on the review. It would be multi-label classification with reviews as features with tokens and conditions as labels with numbers in our training**

In [ ]:
!pip install datasets evaluate transformers sentencepiece

In [28]:
!wget "https://datasets-server.huggingface.co/splits?dataset=forwins%2FDrug-Review-Dataset"

--2024-05-14 12:26:34--  https://datasets-server.huggingface.co/splits?dataset=forwins%2FDrug-Review-Dataset
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 13.226.52.7, 13.226.52.24, 13.226.52.16, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|13.226.52.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272 [application/json]
Saving to: ‘splits?dataset=forwins%2FDrug-Review-Dataset.1’

splits?dataset=forw 100%[===================>]     272  --.-KB/s    in 0s      

2024-05-14 12:26:34 (61.6 MB/s) - ‘splits?dataset=forwins%2FDrug-Review-Dataset.1’ saved [272/272]



In [29]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding, AutoTokenizer

drug_dataset = load_dataset("forwins/Drug-Review-Dataset")
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [31]:
drug_dataset['train']['condition'][:3]

['keratoconjunctivitis sicca', 'birth control', 'birth control']

We need to check how many unique conditions in the our data

In [33]:
checkpoints = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def token_func(data):
    tokenizer(data['review'], truncation=True)

tokenized_data = drug_dataset.map(token_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

Now we have tokens for features, so let's convert the labels to numbers

We are going to delete all of the columns except condition one, it would be used as our labels

In [35]:
encoded_data = encoded_dataset.remove_columns(['patient_id', 'drugName', 'review', 'rating', 'date', 'usefulCount', 'review_length','condition'])

encoded_data.set_format('torch')

encoded_data['train'].column_names

['input_ids', 'token_type_ids', 'attention_mask', 'labels']

In [23]:
encoded_data.set_format('torch')

encoded_data['train'].column_names

['input_ids', 'token_type_ids', 'attention_mask', 'labels']

In [24]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    encoded_data['train'],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    encoded_data['validation'],
    batch_size=8,
    collate_fn=data_collator
)

To quickly examinate the data for errors:

In [25]:
for batch in train_dataloader:
    break
{k: v.shape for k,v in batch.items()}

{'input_ids': torch.Size([8, 180]),
 'token_type_ids': torch.Size([8, 180]),
 'attention_mask': torch.Size([8, 180]),
 'labels': torch.Size([8])}

In [26]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-dbmz-uncased", problem_type="multi_label_classification", num_labels=

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 791]))